In [2]:
import numpy as np
from scipy.signal import convolve2d
import math
from PIL import Image
I = np.array(Image.open(r'./-_000001.tif').convert('L'))
H=np.size(I,0)  #计算 X 的行数
W=np.size(I,1)  #计算 X 的列数

M=np.array([[1, -2, 1], [-2, 4, -2], [1, -2, 1]])
Sigma=sum(sum(abs(convolve2d(I, M))))


Sigma=Sigma*np.sqrt(0.5*math.pi)/(6*(W-2)*(H-2))
print(Sigma)

FileNotFoundError: [Errno 2] No such file or directory: './-_000001.tif'

In [12]:
import os
import sys
import time
from collections.abc import Iterable

import cv2
import hiddenlayer as hl
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from numpy.core.numeric import ones_like
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from torch import nn
from torch.autograd import Variable
import scipy.io as sio
import torchvision.transforms.functional
from ffdnet.models import FFDNet
from ffdnet.test_ffdnet_ipol import ffdnet_vdenoiser
from loss import PSNR, SSIM
from model.model import realFFT, realiFFT
from SwinIR.SwinIR.net import SwinIR
from model.unet import UNet
sys.path.append('..')

In [11]:
def readimg2amp(name, dim=(128,128),out_size=300):
    transform = transforms.Resize(dim)
    amp = Image.open(name)
    amp = np.array(amp.convert('L'))
    amp = amp.astype(np.float32)
    amp = amp / np.max(amp)
    amp = cv2.resize(amp, dim)
    amp = torch.from_numpy(amp)
    H = realFFT(window_size = 2048)
    diffraction_I, _ = H(amp, torch.ones_like(amp), out_size)
    diffraction_I = diffraction_I.unsqueeze(0).unsqueeze(0)
    amp = amp.unsqueeze(0).unsqueeze(0)
    return diffraction_I, amp
def mkdir(path):
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
# I, O =  readimg2amp('ablation/emnist_A4_unet_5_lr=0.001/result/groundtruth.png')
I, O =  readimg2amp('ablation/emnist_A4_unet_5_lr=0.001/result/groundtruth.png')
mkdir('./baselineimg/')
torchvision.utils.save_image(I, './baselineimg/L_I_300.png')
torchvision.utils.save_image(O, './baselineimg/L_O.png')

In [30]:
SNR = 0
S = O.mean()
N = S / 10**(SNR/20)
noise =  torch.randn_like(O) * N
Inoise = O + noise
torchvision.utils.save_image(Inoise, './baselineimg/O_0dB.png')

print(S,N)

tensor(0.1943) tensor(0.1943)


In [20]:
from PIL import Image
import torch
from loss import PSNR, SSIM
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch import nn
import torchvision
from torchvision.transforms import InterpolationMode

import torch
import torch.nn.functional as F
from math import exp
import numpy as np
def findbestposition(O, I_pred, mse) :

    # 定义搜索窗口大小和步长
    window_size = 35  # 搜索窗口大小
    stride = 1  # 步长
    # 初始化最小差异和对应的平移量
    min_diff = float('inf')
    best_translation = (0, 0)
    best_translated_I_pred = I_pred
    # 遍历搜索窗口

    for i in range(-window_size, window_size, stride):
        for j in range(-window_size, window_size, stride):
            # 平移图像
            angle = 0
            translated_I_pred = transforms.functional.affine(I_pred, translate = (i, j), angle=0, scale=1, shear=0) # type: ignore

            # 计算均方误差
            diff = mse(O, translated_I_pred)

            # 如果找到更小的差异，更新最小差异和平移量
            if diff < min_diff:
                min_diff = diff
                best_translation = (i, j)
                best_translated_I_pred = translated_I_pred


    return best_translated_I_pred, min_diff 
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()


def create_window(window_size, channel=1):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
    return window


def ssim(img1, img2, window_size=11, window=None, size_average=True, full=False, val_range=None):
    # Value range can be different from 255. Other common ranges are 1 (sigmoid) and 2 (tanh).
    if val_range is None:
        if torch.max(img1) > 128:
            max_val = 255
        else:
            max_val = 1

        if torch.min(img1) < -0.5:
            min_val = -1
        else:
            min_val = 0
        L = max_val - min_val
    else:
        L = val_range

    padd = 0
    (_, channel, height, width) = img1.size()
    if window is None:
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channel).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=padd, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padd, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=padd, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padd, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padd, groups=channel) - mu1_mu2

    C1 = (0.01 * L) ** 2
    C2 = (0.03 * L) ** 2

    v1 = 2.0 * sigma12 + C2
    v2 = sigma1_sq + sigma2_sq + C2
    cs = v1 / v2  # contrast sensitivity

    ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)

    if size_average:
        cs = cs.mean()
        ret = ssim_map.mean()
    else:
        cs = cs.mean(1).mean(1).mean(1)
        ret = ssim_map.mean(1).mean(1).mean(1)

    if full:
        return ret, cs
    return ret


def msssim(img1, img2, window_size=11, size_average=True, val_range=None, normalize=None):
    device = img1.device
    weights = torch.FloatTensor([0.0448, 0.2856, 0.3001, 0.2363, 0.1333]).to(device)
    levels = weights.size()[0]
    ssims = []
    mcs = []
    for _ in range(levels):
        sim, cs = ssim(img1, img2, window_size=window_size, size_average=size_average, full=True, val_range=val_range)

        # Relu normalize (not compliant with original definition)
        if normalize == "relu":
            ssims.append(torch.relu(sim))
            mcs.append(torch.relu(cs))
        else:
            ssims.append(sim)
            mcs.append(cs)

        img1 = F.avg_pool2d(img1, (2, 2))
        img2 = F.avg_pool2d(img2, (2, 2))

    ssims = torch.stack(ssims)
    mcs = torch.stack(mcs)

    # Simple normalize (not compliant with original definition)
    # TODO: remove support for normalize == True (kept for backward support)
    if normalize == "simple" or normalize == True:
        ssims = (ssims + 1) / 2
        mcs = (mcs + 1) / 2

    pow1 = mcs ** weights
    pow2 = ssims ** weights

    # From Matlab implementation https://ece.uwaterloo.ca/~z70wang/research/iwssim/
    output = torch.prod(pow1[:-1]) * pow2[-1]
    return output
   

In [21]:
def findbestposition(O, I_pred, mse) :

    # 定义搜索窗口大小和步长
    window_size = 35  # 搜索窗口大小
    stride = 1  # 步长
    # 初始化最小差异和对应的平移量
    min_diff = float('inf')
    best_translation = (0, 0)
    best_translated_I_pred = I_pred
    # 遍历搜索窗口

    for i in range(-window_size, window_size, stride):
        for j in range(-window_size, window_size, stride):
            # 平移图像
            angle = 0
            translated_I_pred = transforms.functional.affine(I_pred, translate = (i, j), angle=0, scale=1, shear=0) # type: ignore

            # 计算均方误差
            diff = mse(O, translated_I_pred)

            # 如果找到更小的差异，更新最小差异和平移量
            if diff < min_diff:
                min_diff = diff
                best_translation = (i, j)
                best_translated_I_pred = translated_I_pred


    return best_translated_I_pred, min_diff 

def ssim(img1, img2, window_size=11, window=None, size_average=True, full=False, val_range=None):
    # Value range can be different from 255. Other common ranges are 1 (sigmoid) and 2 (tanh).
    if val_range is None:
        if torch.max(img1) > 128:
            max_val = 255
        else:
            max_val = 1

        if torch.min(img1) < -0.5:
            min_val = -1
        else:
            min_val = 0
        L = max_val - min_val
    else:
        L = val_range

    padd = 0
    (_, channel, height, width) = img1.size()
    if window is None:
        real_size = min(window_size, height, width)
        window = create_window(real_size, channel=channel).to(img1.device)

    mu1 = F.conv2d(img1, window, padding=padd, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padd, groups=channel)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = F.conv2d(img1 * img1, window, padding=padd, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padd, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padd, groups=channel) - mu1_mu2

    C1 = (0.01 * L) ** 2
    C2 = (0.03 * L) ** 2

    v1 = 2.0 * sigma12 + C2
    v2 = sigma1_sq + sigma2_sq + C2
    cs = v1 / v2  # contrast sensitivity

    ssim_map = ((2 * mu1_mu2 + C1) * v1) / ((mu1_sq + mu2_sq + C1) * v2)

    if size_average:
        cs = cs.mean()
        ret = ssim_map.mean()
    else:
        cs = cs.mean(1).mean(1).mean(1)
        ret = ssim_map.mean(1).mean(1).mean(1)

    if full:
        return ret, cs
    return ret


def msssim(img1, img2, window_size=11, size_average=True, val_range=None, normalize=None):
    device = img1.device
    weights = torch.FloatTensor([0.0448, 0.2856, 0.3001, 0.2363, 0.1333]).to(device)
    levels = weights.size()[0]
    ssims = []
    mcs = []
    for _ in range(levels):
        sim, cs = ssim(img1, img2, window_size=window_size, size_average=size_average, full=True, val_range=val_range)

        # Relu normalize (not compliant with original definition)
        if normalize == "relu":
            ssims.append(torch.relu(sim))
            mcs.append(torch.relu(cs))
        else:
            ssims.append(sim)
            mcs.append(cs)

        img1 = F.avg_pool2d(img1, (2, 2))
        img2 = F.avg_pool2d(img2, (2, 2))

    ssims = torch.stack(ssims)
    mcs = torch.stack(mcs)

    # Simple normalize (not compliant with original definition)
    # TODO: remove support for normalize == True (kept for backward support)
    if normalize == "simple" or normalize == True:
        ssims = (ssims + 1) / 2
        mcs = (mcs + 1) / 2

    pow1 = mcs ** weights
    pow2 = ssims ** weights

    # From Matlab implementation https://ece.uwaterloo.ca/~z70wang/research/iwssim/
    output = torch.prod(pow1[:-1]) * pow2[-1]
    return output

In [27]:
file='./figure/noise/0/'
transfrom0 = transforms.ToTensor()
img_name = {}
img={}

img_name[0]=file + 'E_O.png'
img_name[1]=file + 'aCDI_0.png'
img_name[2]=file + 'aHIO_0.png'
for i in range(0,3):
    img[i]= transfrom0(Image.open(img_name[i]).convert('L'))
PSNRval={}
SSIMval={}
mPSNR = PSNR()
mSSIM = SSIM()
bestimg={}
crop = transforms.CenterCrop(128)
mse = nn.MSELoss().cuda()
for i in range(0,3):
    bestimg[i], vmse = findbestposition(img[0], img[i], mse)
    PSNRval[i]= mPSNR(bestimg[i], img[0])
    SSIMval[i]= mSSIM(bestimg[i].unsqueeze(0), img[0].unsqueeze(0))
    ssim_val = ssim(bestimg[i].unsqueeze(0), img[0].unsqueeze(0)) # return (N,)
    ms_ssim_val = msssim(bestimg[i].unsqueeze(0), img[0].unsqueeze(0)) #(N,
    print(img_name[i],'PSNR=%.4g,SSIM=%.4g,VMSE=%.4f,SSIM2=%.4f,ms_ssim=%.4f' %(PSNRval[i],SSIMval[i],vmse,ssim_val,ms_ssim_val))


./figure/noise/0/E_O.png PSNR=inf,SSIM=1,VMSE=0.0000,SSIM2=1.0000,ms_ssim=1.0000
./figure/noise/0/aCDI_0.png PSNR=12.31,SSIM=0.4785,VMSE=0.0587,SSIM2=0.4071,ms_ssim=0.5872
./figure/noise/0/aHIO_0.png PSNR=9.559,SSIM=0.5475,VMSE=0.1107,SSIM2=0.4819,ms_ssim=0.4489
